# Threshold Signature Scheme

In [42]:
from typing import List, Tuple


def fit(points: List[Tuple[int, int]], m: int):
    n = len(points)
    coefficients = [0] * n

    def basis_polynomial(i: int) -> List[int]:
        xi, _ = points[i]
        basis = [1]

        for j, (xj, _) in enumerate(points):
            if i != j:
                # Polynomial multiplication (basis * (x - xj))
                new_basis: List[int] = []
                for k in range(len(basis) + 1):
                    if k == 0:
                        new_basis.append(-xj * basis[k] % m)
                    elif k == len(basis):
                        new_basis.append(basis[k - 1])
                    else:
                        new_basis.append((basis[k - 1] - xj * basis[k]) % m)
                basis = new_basis

                # Divide by (xi - xj) mod m
                inv = pow(xi - xj, -1, m)
                for k in range(len(basis)):
                    basis[k] = basis[k] * inv % m

        return basis

    for i, (_, yi) in enumerate(points):
        li = basis_polynomial(i)
        for j in range(len(li)):
            coefficients[j] = (coefficients[j] + yi * li[j]) % m

    def f(x: int) -> int:
        result = 0
        power_of_x = 1  # x^0 initially
        for coefficient in coefficients:
            result = (result + coefficient * power_of_x) % m
            power_of_x = (power_of_x * x) % m
        return result

    return f

In [43]:
from abc import ABC, abstractmethod
import hashlib
import random
from typing import Any

# TODO: E on ed25519
# from ecpy.curves import Curve, Point
# curve = Curve.get_curve('Ed25519')


def i2b(x: int):
    return x.to_bytes((x.bit_length() + 7) // 8)


class E(ABC):
    @abstractmethod
    def __add__(self, other: Any) -> "E":
        raise NotImplementedError()

    @abstractmethod
    def __mul__(self, other: int) -> "E":
        raise NotImplementedError()

    @abstractmethod
    def __eq__(self, other: Any) -> bool:
        raise NotImplementedError()
    
    @abstractmethod
    def data(self) -> bytes:
        pass

    @property
    @abstractmethod
    def order(self) -> int:
        pass


class MockE(E):
    M: int = pow(2, 31) - 1

    def __init__(self, x: int) -> None:
        self._n = x % self.M
    
    def __add__(self, other: Any) -> "E":
        if not isinstance(other, MockE):
            raise TypeError()
        return MockE(other._n + self._n)

    def __mul__(self, other: int) -> "E":
        return MockE(self._n*other)

    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, MockE):
            return False
        return other._n == self._n
    
    def data(self) -> bytes:
        return i2b(self._n)

    def __repr__(self) -> str:
        return str(self._n)
    
    @property
    def order(self):
        return self.M


def Rhash(x: int, group_public_k: E, message: bytes):
    hasher = hashlib.sha256()
    hasher.update(i2b(x))
    hasher.update(group_public_k.data())
    hasher.update(message)
    return int.from_bytes(hasher.digest(), byteorder='big') % group_public_k.order


def Hhash(R: E, public_k: E, message: bytes):
    assert R.order == public_k.order
    hasher = hashlib.sha256()
    hasher.update(R.data())
    hasher.update(public_k.data())
    hasher.update(message)
    return int.from_bytes(hasher.digest(), byteorder='big') % R.order


class PeerInfo:
    def __init__(self, x: int, group_public_key: E, R_gen: E):
        assert R_gen.order == group_public_key.order
        self._R_gen = R_gen
        self._x = x
        self._group_public_key = group_public_key

    @property
    def x(self):
        return self._x

    @property
    def group_public_key(self):
        return self._group_public_key
        
    def R(self, message: bytes):
        # publicly available calculation
        return self._R_gen * Rhash(self._x, self._group_public_key, message)


class Peer:
    def __init__(self, x: int, private_key: int, group_public_key: E, generator: E):
        assert generator.order == group_public_key.order
        self._generator = generator
        self._x = x
        self._private_key = private_key
        self._salt = random.randint(1, group_public_key.order)
        self._group_public_key = group_public_key
    
    def info(self):
        return PeerInfo(self._x, self._group_public_key, self._generator * self._salt)
    
    def _r(self, message: bytes):
        return self._salt * Rhash(self._x, self._group_public_key, message) % self._generator.order
    
    def sign(self, R_sg: E, message: bytes):
        # private calculation
        r = self._r(message)
        h = Hhash(R_sg, self._group_public_key, message)
        return (r + h * self._private_key) % self._generator.order

## Generating group privates

This needs to be done very secretly since this is the only moment when the private key for the whole group can be calculated

1. Generate a polynomial `f(index) = PrivateKey` from `N` random key points `f = fit([rand, rand for _ in range(N)])`
2. Calculate M (M >= N) private keys like this `PeersPrivateKeys = [f(i+1) for i in range(M)]`
3. Calculate the public key of the group account `GroupPubKey = E(f(0))`
4. Put private keys to peers ensuring secure channel
5. Put group public key to peers ensuring no man in middle
6. Each peer then inside generates a random `Salt` (that does not change from the moment of generation)

In [44]:
def generate_random_points(n: int, order: int):
    points: List[Tuple[int, int]] = []
    for _ in range(n):
        x = random.randint(1, order-1)
        y = random.randint(1, order-1)
        points.append((x, y))
    return points

generator = MockE(1)

N, M = 3, 5
random_points = generate_random_points(N, generator.order)
privates_keys_polynomial = fit(random_points, generator.order)

def genlx(xes: List[int]):
    def lx(_x_i: int, x: int):
        res = 0
        for _x_j in xes:
            if _x_i == _x_j:
                continue
            term = (x - _x_j) * pow(_x_i - _x_j, -1, generator.order) % generator.order
            res = (res + term) % generator.order
        return res
    return lx

group_public_key = generator * privates_keys_polynomial(0)

peers = [Peer(x, privates_keys_polynomial(x), group_public_key, generator) for x in range(1, M+1)]

## Signing

> All operations on raw numbers by modulo of E order

Dealer:
1. Select `N` peers (subset from `M`)
1. Get info from that `N` peers

Peer:
1. Send `R_gen = E(salt)`
1. Send `x, group_public_key`

Dealer:
1. Calculate `Rsg = sum(Rl_i) where Rl_i = R_i*l_i(0), where R_i = R_gen_i * hash(i, GroupPubKey, Message)`
1. Collect signs from peers by sending `Message` and `Rsg` to them

Peer:
1. Calculate `r = salt * hash(i, GroupPubKey, Message)`
1. Calculate `h = hash(Rsg, GroupPubKey, Message)`
1. Send `s = r + h*PrivateKey`

Dealer:
1. Collect `s_i` from `N` peers
1. Calculate the group sign as `gs = fit([(x_i, s_i) for x_i, s_i in zip(xes, collected_signs)])(0)`

In [45]:
# Signing

message = b"Test message for signing"
subgroup = peers[:N]

x = 0

infos = [peer.info() for peer in subgroup]
xes = [info.x for info in infos]
lx = genlx(xes)
R_sg = sum([info.R(message) * lx(info.x, x) for info in infos], generator*0)
signs = [peer.sign(R_sg, message) for peer in subgroup]
gs = fit([(x_i, s_i) for x_i, s_i in zip(xes, signs)], generator.order)(x)

signature = (R_sg, gs)

# Verification
def verify_signature(message: bytes, signature: Tuple[E, int], public_k: E):
    R, s = signature
    h = Hhash(R, public_k, message)
    return R + public_k*h == generator * s

assert verify_signature(message, signature, group_public_key)

AssertionError: 

## Elaborations

The key operation is `gs = fit([(x_i, s_i) for x_i, s_i in zip(xes, collected_signs)])(0)` that effectively utilizes the principles of polynomial interpolation:

Consider a polynomial $f(x)$ that fits a set of points $(x_i, y_i)$. If another polynomial $g(x)$ is constructed such that each point is transformed linearly, where $\hat{y}_i = k \cdot y_i + c_i$, then $g(x)$ can be derived from $f(x)$ using the coefficients $k$ and $c_i$.

This relationship is described by the equation:

$$ g(x) = k \cdot f(x) + \sum c_i \cdot l_i(x) $$

Here, $l_i(x)$ represents the Lagrange basis polynomials. This formulation is valid even within the confines of modular arithmetic.

If $f(x)$ is defined by the private keys of a group and we are interested in a specific point, for instance $f(0)$ (as the representative of the group), the described method shows that setting $r = \sum c_i \cdot l_i(x)$ and $ k = h$ leads to the equation $ s = h \cdot f(0) + r$. This $ s$ corresponds to the valid signature for the "group" public key.

To generate such a signature, N peers must concur on the value of $r$. This consensus is maintained in such a way that the exact value of $r$ remains undisclosed; peers are only aware of its encrypted form. Despite multiple parties agreeing on its encrypted form, extracting $r$ from $s$ requires knowledge of the group's private key, and conversely, obtaining the group's private key explicitly requires $r$ in an unencrypted form.

### Attacks:

1. If hacker got access to N private keys - he can make just the single private key for the entire group